In [1]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os
import numpy as np
import pickle
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
with open('dataset.pkl', 'rb') as f:
    train_set = pickle.load(f)  # (user_id, history, pos, label)
    test_set = pickle.load(f)
    cate_list = pickle.load(f)
    user_count, item_count, cate_count = pickle.load(f)

In [3]:
print(train_set[0:5])
print(test_set[0:5])
print(cate_list)

[(103944, [17704, 37473], 53346, 0), (126219, [15082, 19768, 30450], 48620, 0), (21022, [11725, 4058, 6186, 16970, 13703, 17403, 12736, 30082, 11580, 14475, 32516, 4463, 14413, 25965, 1387, 24154, 48052, 54593, 5021, 32991, 49609, 2111, 10099, 26271, 56329, 15856, 41125, 54526, 51815, 28690], 2179, 0), (134145, [4952, 33036], 55798, 1), (121815, [9906, 30126, 25972, 38025], 45235, 1)]
[(114981, [24265, 8283, 43319, 45475], (56743, 31087)), (50580, [42354, 19191, 13067, 33134, 33800, 58952], (43604, 12255)), (172652, [28839, 32507, 34101, 34339], (49205, 2002)), (98577, [299, 5127, 6140, 14805, 15916, 20120], (17961, 29586)), (147658, [4932, 3674, 10339, 19888], (20257, 45347))]
[738 157 571 ...  63 674 351]


In [4]:
# user_id
user = tf.compat.v1.placeholder(tf.int32, [
    None,
], name='user')
# 正样本的item
pos = tf.compat.v1.placeholder(tf.int32, [
    None,
], name='pos')
# 负样本的item
neg = tf.compat.v1.placeholder(tf.int32, [
    None,
], name='neg')
# label
y = tf.compat.v1.placeholder(tf.float32, [
    None,
], name='y')
# 用户行为特征(User Behavior)中的item序列
hist_i = tf.compat.v1.placeholder(tf.int32, [None, None], name='hist_i')
# User Behavior中序列的真实序列长度
length = tf.compat.v1.placeholder(tf.int32, [
    None,
], name='length')
hidden_units = 128

In [5]:
# user_emb_w = tf.get_variable('user_emb_w',[user_count, hidden_units])
# print(user_emb_w.shape)
# item本身的embedding
item_emb_w = tf.get_variable('item_emb_w', [item_count, hidden_units // 2])
print(item_emb_w.shape)
item_b = tf.get_variable('item_b', [item_count],
                         initializer=tf.constant_initializer(0.0))
# item所属类别的embedding
cate_emb_w = tf.get_variable('cate_emb_w', [cate_count, hidden_units // 2])
print(cate_emb_w.shape)
cate_list = tf.convert_to_tensor(cate_list, dtype=tf.int64)

(63001, 64)
(801, 64)


In [6]:
def attention(queries, keys, keys_length):
    """
        queries:     [B, H]    [batch_size,embedding_size]
        keys:        [B, T, H]   [batch_size,T,embedding_size]
        keys_length: [B]        [batch_size]
        #T为历史行为序列长度
    """
    queries_hidden_units = queries.get_shape().as_list()[-1]  #B*H->H
    queries = tf.tile(queries, [1, tf.shape(keys)[1]])  # B*T
    queries = tf.reshape(
        queries, [-1, tf.shape(keys)[1], queries_hidden_units])  # B*T*H
    din_all = tf.concat([queries, keys, queries - keys, queries * keys],
                        axis=-1)  # B*T*4H
    # 三层全链接(d_layer_3_all为训练出来的atteneion权重）
    d_layer_1_all = tf.layers.dense(din_all,
                                    80,
                                    activation=tf.nn.sigmoid,
                                    name='f1_att')
    d_layer_2_all = tf.layers.dense(d_layer_1_all,
                                    40,
                                    activation=tf.nn.sigmoid,
                                    name='f2_att')
    d_layer_3_all = tf.layers.dense(d_layer_2_all,
                                    1,
                                    activation=None,
                                    name='f3_att') # B*T*1
    #为了让outputs维度和keys的维度一致
    outputs = tf.reshape(d_layer_3_all, [-1, 1, tf.shape(keys)[1]])  # B*1*T
    #  bool类型 tf.shape(keys)[1]为历史行为序列的最大长度，keys_length为人为设定的参数，
    #  如tf.sequence_mask(5，3)  即为array[True,True,True,False,False]
    #  函数的作用是为了后面补齐行为序列，获取等长的行为序列做铺垫
    key_masks = tf.sequence_mask(keys_length, tf.shape(keys)[1])  # B*T
    #在第二维增加一维，也就是由B*T变成B*1*T
    key_masks = tf.expand_dims(key_masks, 1)  # B*1*T
    #tf.ones_like新建一个与output类型大小一致的tensor，设置填充值为一个很小的值，而不是0,padding的mask后补一个很小的负数，这样softmax之后就会接近0
    paddings = tf.ones_like(outputs) * (-2**32 + 1)  # B*1*T
    # key_masks是bool型值，True/False，返回值是对应元素，key_masks中元素为True的元素替换为outputs中的元素，为False的元素替换为paddings中对应元素
    outputs = tf.where(key_masks, outputs, paddings)
    # Scale（缩放）
    outputs = outputs / (keys.get_shape().as_list()[-1]**0.5)
    # Activation
    outputs = tf.nn.softmax(outputs)  # B * 1 * T
    #Weighted Sum outputs=g(Vi,Va)   keys=Vi
    # 这步为公式中的g(Vi*Va)*Vi
    outputs = tf.matmul(
        outputs,
        keys)  # B * 1 * H 三维矩阵相乘，相乘发生在后两维，即 B * (( 1 * T ) * ( T * H ))
    return outputs

In [7]:
def dice(_x, axis=-1, epsilon=0.0000001, name=''):
    alphas = tf.get_variable('alpha' + name,
                             _x.get_shape()[-1],
                             initializer=tf.constant_initializer(0.0),
                             dtype=tf.float32)
    input_shape = list(_x.get_shape())
    reduction_axes = list(range(len(input_shape)))
    del reduction_axes[axis]

    broadcast_shape = [1] * len(input_shape)
    broadcast_shape[axis] = input_shape[axis]

    mean = tf.reduce_mean(_x, axis=reduction_axes)
    broadcast_mean = tf.reshape(mean, broadcast_shape)
    std = tf.reduce_mean(tf.square(_x - broadcast_mean) + epsilon,
                         axis=reduction_axes)
    std = tf.sqrt(std)
    broadcast_std = tf.reshape(std, broadcast_shape)
    # x_normed = (_x - brodcast_mean) / (brodcast_std + epsilon)
    # 以上是手动求均值方差，下面是直接调用bn接口
    x_normed = tf.layers.batch_normalization(_x, center=False, scale=False)
    x_p = tf.sigmoid(x_normed)
    return alphas * (1 - x_p) * _x + x_p * _x

In [8]:
# 从cate_list中取出正样本的cate
pos_cate = tf.gather(cate_list, pos)
# 正样本的embedding，正样本包括item和cate
pos_emb = tf.concat(
    [
        tf.nn.embedding_lookup(item_emb_w, pos),  # (?, 64)
        tf.nn.embedding_lookup(cate_emb_w, pos_cate)  # (?, 64)
    ],
    axis=1)
# 偏置b
pos_b = tf.gather(item_b, pos)
# 从cate_list中取出负样本的cate
neg_cate = tf.gather(cate_list, neg)
# 负样本的embedding，负样本包括item和cate
neg_emb = tf.concat(
    [
        tf.nn.embedding_lookup(item_emb_w, neg),  # (?,64)
        tf.nn.embedding_lookup(cate_emb_w, neg_cate)  # (?,64)
    ],
    axis=1)
# 偏置b
neg_b = tf.gather(item_b, neg)
# 用户行为序列(User Behavior)中的cate序列
hist_cate = tf.gather(cate_list, hist_i)
# 用户行为序列(User Behavior)的embedding，包括item序列和cate序列
hist_emb = tf.concat(
    [
        tf.nn.embedding_lookup(item_emb_w, hist_i),  # (?,?,64)
        tf.nn.embedding_lookup(cate_emb_w, hist_cate)  # (?,?,64)
    ],
    axis=2)
# attention操作
hist = attention(pos_emb, hist_emb, length)
hist = tf.layers.batch_normalization(inputs=hist)
hist = tf.reshape(hist, [-1, hidden_units])
hist = tf.layers.dense(hist, hidden_units)
u_emb = hist

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
(?, ?, 1)
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).


In [9]:
# 下面两个全连接用来计算y'，i为正样本，j为负样本
din_pos = tf.concat([u_emb, pos_emb], axis=-1)
din_pos = tf.layers.batch_normalization(inputs=din_pos, name='b1')
d_layer_1_pos = tf.layers.dense(din_pos, 80, activation=None, name='f1')
d_layer_1_pos = dice(d_layer_1_pos, name='dice_1_pos')
d_layer_2_pos = tf.layers.dense(d_layer_1_pos, 40, activation=None, name='f2')
d_layer_2_pos = dice(d_layer_2_pos, name='dice_2_pos')
d_layer_3_pos = tf.layers.dense(d_layer_2_pos, 1, activation=None, name='f3')

In [10]:
din_neg = tf.concat([u_emb, neg_emb], axis=-1)
din_neg = tf.layers.batch_normalization(inputs=din_neg, name='b1', reuse=True)
d_layer_1_neg = tf.layers.dense(din_neg,
                                80,
                                activation=None,
                                name='f1',
                                reuse=True)
d_layer_1_neg = dice(d_layer_1_neg, name='dice_1_neg')
d_layer_2_neg = tf.layers.dense(d_layer_1_neg,
                                40,
                                activation=None,
                                name='f2',
                                reuse=True)
d_layer_2_neg = dice(d_layer_2_neg, name='dice_2_neg')
d_layer_3_neg = tf.layers.dense(d_layer_2_neg,
                                1,
                                activation=None,
                                name='f3',
                                reuse=True)

d_layer_3_pos = tf.reshape(d_layer_3_pos, [-1])
d_layer_3_neg = tf.reshape(d_layer_3_neg, [-1])

In [11]:
#预测的（y正-y负）
x = pos_b - neg_b + d_layer_3_pos - d_layer_3_neg
#预测的（y正）
logits = pos_b + d_layer_3_pos

In [12]:
# u_emb_all = tf.expand_dims(u_emb, 1)
# u_emb_all = tf.tile(u_emb_all, [1, item_count, 1])
# all_emb = tf.concat([item_emb_w, tf.nn.embedding_lookup(cate_emb_w, cate_list)], axis=1)
# all_emb = tf.expand_dims(all_emb, 0)
# all_emb = tf.tile(all_emb, [512, 1, 1])
# din_all = tf.concat([u_emb_all, all_emb], axis=-1)
# din_all = tf.layers.batch_normalization(inputs=din_all, name='b1', reuse=True)
# d_layer_1_all = tf.layers.dense(din_all, 80, activation=None, name='f1', reuse=True)
# d_layer_1_all = dice(d_layer_1_all, name='dice_1_all')
# d_layer_2_all = tf.layers.dense(d_layer_1_all, 40, activation=None, name='f2', reuse=True)
# d_layer_2_all = dice(d_layer_2_all, name='dice_2_all')
# d_layer_3_all = tf.layers.dense(d_layer_2_all, 1, activation=None, name='f3', reuse=True)
# d_layer_3_all = tf.reshape(d_layer_3_all, [-1, item_count])

In [17]:
# logits_all = tf.sigmoid(item_b+d_layer_3_all)
mf_auc = tf.reduce_mean(tf.to_float(x > 0))
score_pos = tf.sigmoid(pos_b + d_layer_3_pos)
score_neg = tf.sigmoid(neg_b + d_layer_3_neg)
score_pos = tf.reshape(score_pos, [-1, 1])
score_neg = tf.reshape(score_neg, [-1, 1])
p_and_n = tf.concat([score_pos, score_neg], axis=-1)
# tf.summary.scalar("auc", mf_auc)
loss = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y))
tf.summary.scalar("loss", loss)

<tf.Tensor 'loss:0' shape=() dtype=string>

In [18]:
def get_batch_data(i, batch_size):
    start = j * batch_size
    end = (j + 1) * batch_size
    end = end if end < len(train_set) else len(train_set)
    train_batch = train_set[start:end]
    user_batch = [i[0] for i in train_batch]
    pos_batch = [i[2] for i in train_batch]  # 本次点击的样本id
    y_batch = [i[3] for i in train_batch
               ]  # 本次点击id是否存在于用户历史点击序列中，负样本是自己构造的，肯定不在用户点击历史中
    length_batch = [len(i[1]) for i in train_batch]
    hist_i_batch = np.zeros(
        [len(train_batch), max(length_batch)], np.int64)  # 用户点击历史
    k = 0
    for t in train_batch:
        for l in range(len(t[1])):
            hist_i_batch[k][l] = t[1][l]
        k += 1
    return (user_batch, pos_batch, y_batch, hist_i_batch, length_batch)

In [19]:
def get_test_batch_data(i, batch_size):
    start = j * batch_size
    end = (j + 1) * batch_size
    end = end if end < len(test_set) else len(test_set)
    test_batch = test_set[start:end]
    user_batch = [i[0] for i in test_batch]
    pos_batch = [i[2][0] for i in test_batch]  # 正样本
    neg_batch = [i[2][1] for i in test_batch]  # 负样本
    length_batch = [len(i[1]) for i in test_batch]
    hist_i_batch = np.zeros(
        [len(test_batch), max(length_batch)], np.int64)  # 用户点击历史
    k = 0
    for t in test_batch:
        for l in range(len(t[1])):
            hist_i_batch[k][l] = t[1][l]
        k += 1
    return (user_batch, pos_batch, neg_batch, hist_i_batch, length_batch)

In [20]:
train_op = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(loss)
epoches = 20
batch_size = 128
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter("/data00/home/huangyajian/code/tf_log/",
                                   sess.graph)
    for i in range(epoches):
        batch = (int)(len(train_set) / batch_size)
        for j in range(batch):
            user_batch, pos_batch, y_batch, hist_i_batch, length_batch = get_batch_data(
                j, batch_size)
            l, _ = sess.run(
                [loss, train_op],
                feed_dict={
                    user: user_batch,
                    pos: pos_batch,
                    hist_i: hist_i_batch,
                    y: y_batch,
                    length: length_batch
                })
            print("loss: ", l)
            if j % 100 == 0:
                #                 user_batch_test, pos_batch_test, neg_batch_test, hist_i_batch_test, length_batch_test = get_test_batch_data(j, batch_size)
                #                 rs = sess.run(
                #                     [merged],
                #                     feed_dict={
                #                         user: user_batch_test,
                #                         pos: pos_batch_test,
                #                         neg: neg_batch_test,
                #                         hist_i: hist_i_batch_test,
                #                         length: length_batch_test
                #                     })
                #                 writer.add_summary(rs, j)
                rs = sess.run(merged,
                              feed_dict={
                                  user: user_batch,
                                  pos: pos_batch,
                                  hist_i: hist_i_batch,
                                  y: y_batch,
                                  length: length_batch
                              })
                writer.add_summary(rs, j)

loss:  0.6932454
loss:  0.6935494
loss:  0.6903191
loss:  0.69957167
loss:  0.68729526
loss:  0.7010373
loss:  0.69755274
loss:  0.69126606
loss:  0.6945267
loss:  0.69850993
loss:  0.6907612
loss:  0.6931665
loss:  0.6852367
loss:  0.6977018
loss:  0.6953786
loss:  0.70441073
loss:  0.68979686
loss:  0.69872284
loss:  0.69000983
loss:  0.68639374
loss:  0.6987846
loss:  0.6899035
loss:  0.6900839
loss:  0.6907774
loss:  0.7016809
loss:  0.69172144
loss:  0.6969315
loss:  0.6931871
loss:  0.6912251
loss:  0.6898962
loss:  0.69575423
loss:  0.6948594
loss:  0.6893753
loss:  0.703921
loss:  0.69313335
loss:  0.6937824
loss:  0.69240403
loss:  0.6931088
loss:  0.6939084
loss:  0.69281924
loss:  0.6936897
loss:  0.69431525
loss:  0.69107765
loss:  0.7019503
loss:  0.6933042
loss:  0.69249773
loss:  0.6921172
loss:  0.692208
loss:  0.69221187
loss:  0.6960041
loss:  0.69264174
loss:  0.6931572
loss:  0.6903663
loss:  0.6955194
loss:  0.6919825
loss:  0.69635665
loss:  0.6920087
loss:  0.691

KeyboardInterrupt: 